In [2]:
import sys
import pickle
sys.path.append('/home/ubuntu/w210_melanoma/melanoma-detection/')
with open('/home/ubuntu/preprocessor.pkl', 'rb') as f:
    preprocessor = pickle.load(f)

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [5]:
df = pd.read_csv(
    '/home/ubuntu/w210_melanoma/melanoma-detection/concatenated_whole.csv.gz', 
    compression='gzip',
    error_bad_lines=False,
)

In [6]:
drop_col = ['image_name','patient_id', 'diagnosis',	'benign_malignant']
data = df.drop(drop_col,axis =1)

In [8]:
train, test = train_test_split(data, test_size=0.2, random_state=0)

In [9]:
X_train = train.drop(['target'], axis=1)
y_train = train['target']
X_test = test.drop(['target'], axis=1)
y_test = test['target']

In [10]:
X_train_trasform = preprocessor.transform(X_train)
X_train_fin = pd.DataFrame(X_train_trasform)
X_test_trasform = preprocessor.transform(X_test)
X_test_fin = pd.DataFrame(X_test_trasform)

In [12]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [13]:
def make_model(metrics=METRICS, output_bias=None):
    if output_bias is not None:
        output_bias = keras.initializers.Constant(output_bias)
    model = keras.Sequential([
        keras.layers.Dense(32, activation='relu', input_shape=(2059,)),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(8, activation='relu'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias),
  ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics)

    return model

In [14]:
EPOCHS = 1000
BATCH_SIZE = 1024

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=200,
    mode='max',
    restore_best_weights=True)

In [15]:
neg, pos = np.bincount(y_train)
initial_bias = np.log([pos/neg])

In [16]:
model = make_model(output_bias=initial_bias)

Extension horovod.torch has not been built: /home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-37m-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2021-11-18 22:03:19.815 ip-172-31-94-192:2438 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-11-18 22:03:19.836 ip-172-31-94-192:2438 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


In [17]:
class_weight = {0: 1, 1: 100}

In [18]:
history = model.fit(
    X_train_fin,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    validation_data=(X_test_fin, y_test),
    class_weight=class_weight,
)

Epoch 1/1000
26/26 [==============================] - 3s 51ms/step - loss: 4.4731 - tp: 56.8889 - fp: 3137.0000 - tn: 10907.7037 - fn: 187.2963 - accuracy: 0.8349 - precision: 0.0121 - recall: 0.1558 - auc: 0.5194 - prc: 0.0169 - val_loss: 0.5594 - val_tp: 1.0000 - val_fp: 52.0000 - val_tn: 6459.0000 - val_fn: 114.0000 - val_accuracy: 0.9749 - val_precision: 0.0189 - val_recall: 0.0087 - val_auc: 0.6426 - val_prc: 0.0286
Epoch 2/1000
26/26 [==============================] - 0s 15ms/step - loss: 3.1036 - tp: 124.8148 - fp: 6432.9630 - tn: 7602.2593 - fn: 128.8519 - accuracy: 0.5310 - precision: 0.0187 - recall: 0.5056 - auc: 0.5186 - prc: 0.0179 - val_loss: 0.4406 - val_tp: 0.0000e+00 - val_fp: 8.0000 - val_tn: 6503.0000 - val_fn: 115.0000 - val_accuracy: 0.9814 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.6847 - val_prc: 0.0339
Epoch 3/1000
26/26 [==============================] - 0s 15ms/step - loss: 3.0017 - tp: 125.3333 - fp: 6511.2963 - tn: 7522.6296 - fn: 129.

In [19]:
model.save('/home/ubuntu/w210_melanoma/model/NN_with_pickled_preprocessor.hdf5')